In [1]:
import cv2
import numpy 
import copy
from scipy.spatial import distance
import random
import matplotlib.pyplot as plt

def Init_Board():
    
    img_size = [500,500]
    img = numpy.zeros([500,500,3])

    img[:,:,0] = numpy.ones(img_size)*240.0
    img[:,:,1] = numpy.ones(img_size)*240.0
    img[:,:,2] = numpy.ones(img_size)*240.0

    grid_size = int(input("Give the Size of the Grid   "))
    #grid_size = 3
    vertices = []
    centre_squares = []
    for i in range(grid_size + 1):
        for j in range(grid_size + 1):
            if grid_size%2 == 0:
                img = cv2.circle(img,((250 + 50*int(grid_size/2)) - i*50,(250 + 50*int(grid_size/2)) - j*50), 3, (0,0,0), -1)
                vertices.append(((250 + 50*int(grid_size/2)) - i*50,(250 + 50*int(grid_size/2)) - j*50))
            else:
                img = cv2.circle(img,((250 + 50*int(grid_size/2)+25) - i*50,(250 + 50*int(grid_size/2)+25) - j*50), 3, (0,0,0), -1)
                vertices.append(((250 + 50*int(grid_size/2)+25) - i*50,(250 + 50*int(grid_size/2)+25) - j*50))

    for i in vertices:
        m,n = i[0]+25,i[1]+25
        if m < (250 + 50*int(grid_size/2)+25) and n < (250 + 50*int(grid_size/2)+25):
            centre_squares.append((m,n))
            #img = cv2.circle(img,(m,n), 3, (0,0,255), -1)            

    img_clone = img.copy()

    for i in vertices:
        #print(i)
        img = cv2.circle(img,i, 9, (0,0,0), -1)   
    #print(vertices)

    img = cv2.addWeighted(img_clone, 0.7, img, 0.3, 0)

    cv2.imwrite('Dots_Lines.jpg', img)
    
    return vertices,centre_squares,grid_size

In [2]:
vertices,centre_squares = Init_Board()

In [3]:
def check_squares(centre_squares,colour):
    img = cv2.imread('Dots_Lines.jpg')
    for i in centre_squares:
        p,q = i
        
        x1 = p - 25
        x2 = p + 25
        y1 = q - 25
        y2 = q + 25
        if colour == 'blue':
            if sum(img[x1][q]) in range(0,700) and sum(img[x2][q]) in range(0,700) and sum(img[p][y1]) in range(0,700) and sum(img[p][y2]) in range(0,700) and sum(img[p][q]) == 720:
                pts = numpy.array([[y1,x1],[y1,x2],[y2,x2],[y2,x1]], numpy.int32)
                img = cv2.fillPoly(img,[pts],(255,204,153))
                cv2.imwrite('Dots_Lines.jpg', img)
                #centre_squares.remove((p,q))
                return True
        elif colour == 'red':
            if sum(img[x1][q]) in range(0,700) and sum(img[x2][q]) in range(0,700) and sum(img[p][y1]) in range(0,700) and sum(img[p][y2]) in range(0,700) and sum(img[p][q]) == 720:
                pts = numpy.array([[y1,x1],[y1,x2],[y2,x2],[y2,x1]], numpy.int32)
                img = cv2.fillPoly(img,[pts],(204,204,255))
                cv2.imwrite('Dots_Lines.jpg', img)
                #centre_squares.remove((p,q))
                return True
        

In [4]:
def find_adjacent(x,y):
    
    L = set()

    x1 = x - 50
    x2 = x + 50
    y1 = y - 50
    y2 = y + 50
    
    [L.add((item[0],y)) for item in vertices if item[0] == x1]
    [L.add((item[0],y)) for item in vertices if item[0] == x2]
    [L.add((x,item[1])) for item in vertices if item[1] == y1]
    [L.add((x,item[1])) for item in vertices if item[1] == y2]
    
    #print('Adj')
    #p = vertices
#     for i in range(len(p)):
#         if p[i][0] == x1:
#             L.add((x1,y))
    
#     for i in range(len(p)):
#         if p[i][0] == x2:
#             L.add((x2,y))
        
#     for i in range(len(p)):
#         if p[i][1] == y1:
#             L.add((x,y1))
        
#     for i in range(len(p)):
#         if p[i][1] == y2:
#             L.add((x,y2))
        
    return L

In [5]:
def draw_line(start,end,colour):
    img = cv2.imread('Dots_Lines.jpg')
    if colour == 'blue':
        img = cv2.line(img,start,end,(204,102,0),3)
        img = cv2.circle(img,start, 3, (0,0,0), -1)
        img = cv2.circle(img,end, 3, (0,0,0), -1)
        cv2.imwrite('Dots_Lines.jpg', img)
    elif colour == 'red':
        img = cv2.line(img,start,end,(0,0,204),3)
        img = cv2.circle(img,start, 3, (0,0,0), -1)
        img = cv2.circle(img,end, 3, (0,0,0), -1)
        cv2.imwrite('Dots_Lines.jpg', img)
        

In [6]:
def check_game(centre_squares):
    img = cv2.imread('Dots_Lines.jpg')
    flag = True
    for i in centre_squares:
        p,q = i
        
        if sum(img[p][q]) == 720:
            flag = True
            break
        else:
            flag = False
    return flag
    

In [7]:
def Player_Turn(grid_size):
    points = []
    
    if grid_size == 2:
        X = ((input("Give a Starting Node x in range(0,2) and y in range(0,2)   ")).split(','))
        gx1 = 200 + 50*int(X[0])
        gy1 = 200 + 50*int(X[1])
        points.append((gx1,gy1))
    elif grid_size == 3:
        X = ((input("Give a Starting Node x in range(0,3) and y in range(0,3)   ")).split(','))
        gx1 = 175 + 50*int(X[0])
        gy1 = 175 + 50*int(X[1])
        points.append((gx1,gy1))
    
    Adj = find_adjacent(gx1,gy1)
    
    gx2,gy2 = -1,-1
    while (gx2,gy2) not in Adj:
        if grid_size == 2:
            X = ((input("Give a Ending Node x in range(0,2) and y in range(0,2) ")).split(','))
            gx2 = 200 + 50*int(X[0])
            gy2 = 200 + 50*int(X[1])
        elif grid_size == 3:
            X = ((input("Give a Ending Node x in range(0,3) and y in range(0,3) ")).split(','))
            gx2 = 175 + 50*int(X[0])
            gy2 = 175 + 50*int(X[1])
        
    points.append((gx2,gy2))
    return points

In [8]:
def choose_first():
    return random.choice(['blue','red'])

In [9]:
def players_colour():
    
    P1_marker = ''
    P2_marker = ''
    
    while P1_marker not in ('red','blue'):
        P1_marker = input("Please pick a colour 'red' or 'blue' ")
        
    if P1_marker == 'red':
        P2_marker = 'blue'
        print(f"player1 has choose {P1_marker} and so player2 is {P2_marker}")
        return ('red','blue')
    else:
        P2_marker = 'red'
        print(f"player1 has choose {P1_marker} and so player2 is {P2_marker}")
        return ('blue','red')
        
    

In [10]:
def replay():
    
    replay_input = input("Do you want to play again. Say yes or no: ")
    
    while replay_input == 'yes' or replay_input == 'no':
        if replay_input == 'yes':
            return True
        elif replay_input == 'no':
            return False

In [11]:
class Player:
    
    def __init__(self, colour, score = 0,turn = None):
        self.colour = colour
        self.score = score
        self.turn = turn

In [12]:
def Play_Game():
    vertices,centre_squares,grid_size = Init_Board()
    while True:
        print('player1 choose your colour')
        P1_colour,P2_colour = players_colour()

        P1 = Player(P1_colour)
        P2 = Player(P2_colour)

        first_player = choose_first()
        print(f'{first_player} will go first, randomly choosen') 
        if P1.colour == first_player:
            P1.turn = True
            P2.turn = False
        else:
            P1.turn = False
            P2.turn = True

        seen = set()
        while check_game(centre_squares):

            if P1.turn == True:
                P1.turn = False
                P2.turn = True

                print(f'{P1_colour} choose your move')
                line = Player_Turn(grid_size)
                if str(line) not in seen:
                    seen.add(str(line))
                    draw_line(line[0],line[1],P1.colour)
                else:
                    P1.turn = True
                    P2.turn = False
                    print("Already Occupied")

                if check_squares(centre_squares,P1.colour):
                    P1.score += 1
                    P1.turn = True
                    P2.turn = False
                    if check_squares(centre_squares,P1.colour):
                        P1.score += 1
                        P1.turn = True
                        P2.turn = False



            elif P2.turn == True:
                P1.turn = True
                P2.turn = False

                print(f'{P2_colour} choose your move')
                line = Player_Turn(grid_size)
                if str(line) not in seen:
                    seen.add(str(line))
                    draw_line(line[0],line[1],P2.colour)
                else:
                    P1.turn = False
                    P2.turn = True
                    print("Already Occupied")

                if check_squares(centre_squares,P2.colour):
                    P2.score += 1
                    P1.turn = False
                    P2.turn = True
                    if check_squares(centre_squares,P2.colour):
                        P2.score += 1
                        P1.turn = False
                        P2.turn = True

        if P1.score > P2.score:
            print('P1 Score',P1.score)
            print('P2 Score',P2.score)
            print(f'{P1.colour} has won the game')
        elif P2.score > P1.score:
            print('P1 Score',P1.score)
            print('P2 Score',P2.score)
            print(f'{P2.colour} has won the game')
        else:
            print('P1 Score',P1.score)
            print('P2 Score',P2.score)
            print('Game Tied')

        if replay():
            continue
        else:
            print('Good Bye')

        break


In [13]:
Play_Game()

player1 choose your colour
Please pick a colour 'red' or 'blue' blue
player1 has choose blue and so player2 is red
blue will go first, randomly choosen
blue choose your move
Give a Starting Node x in range(0,3) and y in range(0,3)   0,0
Give a Ending Node x in range(0,3) and y in range(0,3) 1,0
red choose your move
Give a Starting Node x in range(0,3) and y in range(0,3)   3,3
Give a Ending Node x in range(0,3) and y in range(0,3) 3,2
blue choose your move


KeyboardInterrupt: 